# Bag of Words

Step 1: Tokenize the data, remove stop words and perform stemming or lemmanization

In [2]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omv-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Error loading omv-1.4: Package 'omv-1.4' not found in
[nltk_data]     index


False

In [15]:
paragraph = "The news mentioned here is fake fake. Audience do not encourage fake news. Fake news is false or misleading"



sentences = nltk.sent_tokenize (paragraph)

print(sentences)

lemmatizer = WordNetLemmatizer()

print(lemmatizer)



corpus = []



for i in range(len(sentences)):

    sent = re.sub('[^a-zA-Z]',' ', sentences[i])

    print(sent)

    sent = sent.lower()
    sent = sent.split()

    stop_wrd = set(stopwords.words('english'))

    #print(stop_wrd)



    sent = [lemmatizer.lemmatize (word) for word in sent if not word in stop_wrd]

    print(sent)
    sent = ' '.join(sent)
    corpus.append(sent)
print(corpus)




['The news mentioned here is fake fake.', 'Audience do not encourage fake news.', 'Fake news is false or misleading']
<WordNetLemmatizer>
The news mentioned here is fake fake 
['news', 'mentioned', 'fake', 'fake']
Audience do not encourage fake news 
['audience', 'encourage', 'fake', 'news']
Fake news is false or misleading
['fake', 'news', 'false', 'misleading']
['news mentioned fake fake', 'audience encourage fake news', 'fake news false misleading']


### Step 2: List all unique words

Unique words: [‘news’, ‘mentioned’, ‘fake’, ‘audience’, ‘encourage’, ‘false’, ‘misleading’]

### Step 3: Create a dictionary with mapping of words to a number. This should be sorted on frequency of occurrence in descending order.

### Step 4: Create a table in each sentence, for the presence of each word in the dictionary, assign ‘1’ else assign ‘0’

Therefore, the transformed data are :

news mentioned fake — [0 0 1 0 1 0 1]

audience encourage fake news — [1 1 1 0 0 0 1]

fake news false misleading — [0 0 1 1 0 1 1]


In [4]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
independentFeature = cv.fit_transform(corpus).toarray()
print(independentFeature)

[[0 0 2 0 1 0 1]
 [1 1 1 0 0 0 1]
 [0 0 1 1 0 1 1]]


Text Representation

In [7]:
import pandas as pd
import numpy as np

In [8]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount= True)

Mounted at /content/gdrive/


In [9]:

df=pd.read_csv("/content/gdrive/My Drive/ML_practical/NLP/spam.csv")
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [10]:
df.Category.value_counts()
df['spam'] = df['Category'].apply(lambda x: 1 if x =='spam' else 0)
df.shape
df.head()


,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.Message, df.spam, test_size=0.2)


In [12]:
from sklearn.feature_extraction.text import CountVectorizer

v = CountVectorizer()

X_train_cv = v.fit_transform(X_train.values)
X_train_cv
X_train_cv.toarray()[:2][0]
v.vocabulary_
X_train_np = X_train_cv.toarray()
X_train_np[0]


array([0, 0, 0, ..., 0, 0, 0])

In [13]:

from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_train_cv, y_train)
X_test_cv = v.transform(X_test)
from sklearn.metrics import classification_report

y_pred = model.predict(X_test_cv)

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.98      1.00      0.99       971
           1       0.98      0.89      0.93       144

    accuracy                           0.98      1115
   macro avg       0.98      0.94      0.96      1115
weighted avg       0.98      0.98      0.98      1115

